In [1]:
import logging
import os
import re
from urllib.parse import urlparse, parse_qs
import tempfile


from bs4 import BeautifulSoup
import jinja2
import requests

from tessa_chef import make_request, get_parsed_html_from_url, make_fully_qualified_url
from tessa_chef import get_license, get_text
from tessa_chef import get_section_filename, create_predictable_zip, download_file
from tessa_chef import shutil



from le_utils.constants import content_kinds, file_formats, licenses

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)


## Test download_module funcion


In [2]:
# from tessa_chef import download_module
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108' # EN
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82912' # AR
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR

# toc = download_module(test_module)


In [3]:
# prints TOC
# for section in toc['children']:
#     print('  - section:', section['title'], section['filename'])
#     for subsection in section['children']:
#         print('    - subsection:', subsection['title'], subsection['filename'])




## Test scraping


In [3]:
from tessa_chef import TessaChef
tessa_chef = TessaChef()
tessa_chef.scrape(None, {'lang':'fr'})

Scrapring module @ url = http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=105334
destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpdrsqxcwf


AttributeError: 'NoneType' object has no attribute 'find_parent'

In [2]:

from tessa_chef import urlparse, parse_qs, get_parsed_html_from_url
from tessa_chef import _get_next_section_url, download_page, download_module_no_toc


test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR
download_module_no_toc(test_module, lang='fr')

destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpihwglq_b
Scrapring section/subsectino... section-1.html
  - section: Module 1 : Étude du nombre et de la structure
Scrapring section/subsectino... section-1_1.html
    - subsection: Module 1 : Étude du nombre et de la structure: 1. Utilisation de jeux pour développer le calcul mental
Scrapring section/subsectino... section-1_2.html
    - subsection: Module 1 : Étude du nombre et de la structure: 2. Identifier les mathématiques pratiquées dans le jeu – Jouer en groupes
Scrapring section/subsectino... section-1_3.html
    - subsection: Module 1 : Étude du nombre et de la structure: 3. Utiliser des jeux culturels traditionnels
Scrapring section/subsectino... section-1_4.html
    - subsection: Module 1 : Étude du nombre et de la structure: Ressource 1: Jeux d’associations de nombres
Scrapring section/subsectino... section-1_5.html
    - subsection: Module 1 : Étude du nombre et de la structure: Ressource 2: Jeux utilisant des

'/var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpxkkfwwka.zip'

In [2]:
import os
os.getcwd()

'/Users/ivan/Projects/FLECode/sushi-chef-tessa/notebooks'

In [ ]:
os.path.